In [ ]:
# Gene ontology enrichment analysis of eye size-associated genes across deepwater Malawi cichlids of the genus Diplotaxodon
# Original manuscript: "Widespread genetic signals of visual system adaptation in deepwater cichlid fishes"

# Eye size GWAS outliers defined as "top 0.01% associations"
# Included in GO analysis: genes with transcription start site within 25 kb distance from outlier SNPs

# 1. Packages installation

In [ ]:
# Install Bioconductor
if(!requireNamespace("BiocManager", quietly=TRUE)) {
  install.packages("BiocManager")}

In [ ]:
# Install topGO package
BiocManager::install("topGO")

In [ ]:
BiocManager::install("org.Dr.eg.db")    # Zebrafish annotation package from Bioconductor

In [ ]:
# Load packages
library(topGO)
library(org.Dr.eg.db)

In [ ]:
packageVersion('topGO')

# 2. Input data

## 2.1. Path to files

In [ ]:
# 'Gene universe'
all_genes_mix_fn <- "../data/03_GO_genes_top001pct_25kbrange_ENSEMBL-genenames.txt"

# Test genes
test_genes_mix_ids_25kb_fn <- "../data/03_GO_genes_GWAS_callset_ENSEMBL-genenames.txt"

## 2.2. Read in data

In [ ]:
## All genes
all_genes_mix <- read.csv(all_genes_mix_fn, header=FALSE)

## Test genes
test_genes_ensembl <- read.csv(test_genes_ensembl_fn, header=FALSE)

# 3. Analysis

## 3.1. topGO object

In [ ]:
# Function that creates GO data object for further analyses
make_GO_object <- function(description,AllGenes_path, GenesOfInterest_path,
                           ontology, nodeSize) {
    
    # Create gene universe
    all_genes <- read.csv(AllGenes_path, header=FALSE)
    gene_universe <- as.vector(all_genes$V1)
    #gene_universe <- gene_universe[!duplicated(gene_universe)]   # remove duplicates
    
    # Read in list of interesting genes to test
    test_genes <- as.vector(read.csv(GenesOfInterest_path,
                       sep = ',',header = FALSE)[[1]]) 
    
    # Classify genes into test ('1') and no test ('0'). This will be the input data for GOdata object.
    geneClassify <- factor(as.integer(gene_universe %in% test_genes))
    names(geneClassify) <- gene_universe
    str(geneClassify)
    
    # Build GOdata object    
    GOdata <- new("topGOdata", description = description, ontology = ontology,
                  allGenes = geneClassify,
                  nodeSize = nodeSize,
                  annot = annFUN.org, mapping = "org.Dr.eg.db", ID = "symbol")   # using zebrafish database for mapping of GO terms
}

In [ ]:
# Biological process
GO_BP <- make_GO_object(description="genes around 25 kb of top 0.01% sig GWAS - mixed ensembl & gene names",
                        all_genes_mix_fn, test_genes_mix_ids_25kb_fn, ontology="BP", nodeSize=5)

In [ ]:
# Molecular function
GO_MF <- make_GO_object(description="genes around 25 kb of top 0.01% sig GWAS - mixed ensembl & gene names",
                        all_genes_mix_fn, test_genes_mix_ids_25kb_fn, ontology="MF", nodeSize=5)

## Fisher's exact test

In [ ]:
# Using different algorithms to account for dependency of GO categories
resultClassic <- runTest(GO_BP, algorithm = "classic", statistic = "fisher")
resultElim <- runTest(GO_BP, algorithm = "elim", statistic = "fisher")
resultWeight <- runTest(GO_BP, algorithm = "weight", statistic = "fisher")

In [ ]:
# Same for molecular function
resultClassic1 <- runTest(GO_MF, algorithm = "classic", statistic = "fisher")
resultElim1 <- runTest(GO_MF, algorithm = "elim", statistic = "fisher")
resultWeight1 <- runTest(GO_MF, algorithm = "weight", statistic = "fisher")

In [ ]:
# e.g., BP Fischer test results weight algorithm
resultWeight

## Results

In [ ]:
# BP weight
FT_resultWeight <- GenTable(GO_BP, countFisher = resultWeight,
                            orderBy = "countFisher",
                            topNodes = length(score(resultWeight))) 

In [ ]:
# All three tests in one table
allRes <- GenTable(GO_BP, 
                   classic = resultClassic, elim = resultElim, weight = resultWeight,
                   orderBy = "weight",
                   ranksOf = "classic", 
                   topNodes = length(score(resultWeight)))

In [ ]:
# Add lists of genes to each GO term
test_genes25kb <- as.vector(read.csv(test_genes_mix_ids_25kb_fn,
                                     sep = ',',
                                     header = FALSE)[[1]]) 

In [ ]:
FT_resultWeight$genes <- sapply(FT_resultWeight$GO.ID, function(x)
    {genes <- genesInTerm(GO1_BP, x)
     genes[[1]][genes[[1]] %in% test_genes25kb] # myGenes is the queried gene list
    })

FT_resultWeight$genes[which(FT_resultWeight$countFisher<0.01)] # print those only with p-value < 0.01

In [ ]:
# Print table
FT_resultWeight

In [ ]:
# Export results table
FT_resultWeight[, 7] <- paste(FT_resultWeight[, 7])
colnames(FT_resultWeight)[7] <- paste(colnames(FT_resultWeight)[7], collapse = ",")

write.csv(FT_resultWeight, file = "outfile.csv", sep = "\t", row.names = FALSE)